# Day27 BS4 Scrape from Youtube 1/2 
# 用美麗的湯爬取Youtube 1/2

今天嘗試來用美麗的湯從Youtube爬取影片標題、連結、觀看次數與簡介。爬取的資料是一個好聽德國樂團[Berge](http://www.hoertberge.de/)的Youtube搜尋頁面(是想趁機推坑吧笑死)。<br>
Today we are going to scrape title, link, view counts, and descriptions from Youtube using Beautiful Soup. The url we are scraping today is the search result on Youtube of a German band [Berge](http://www.hoertberge.de/).<br>
![Title](2701.JPG)

In [18]:
# 同前兩天的起始步驟，載入套件、創建美麗湯物件
# Same set up steps as in day25, import packages, set up a beautiful soup object
import requests
from bs4 import BeautifulSoup

url = "https://www.youtube.com/results?search_query=Berge"
request = requests.get(url)
content = request.content
soup = BeautifulSoup(content, "html.parser")
print(soup) # 看一下怎麼區分的 have a peak of how each video's seperated

 <!DOCTYPE html>
<html data-cast-api-enabled="true" lang="zh-TW"><head><style name="www-roboto">@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:local('Roboto Medium'),local('Roboto-Medium'),url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:local('Roboto Regular'),local('Roboto-Regular'),url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:local('Roboto Italic'),local('Roboto-Italic'),url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:italic;font-weight:500;src:local('Roboto Medium Italic'),local('Roboto-MediumItalic'),url(//fonts.gstatic.com/s/roboto/v18/KFOjCnqEu92Fr1Mu51S7ACc6CsE.ttf)format('truetype');}</style><script name="www-roboto">if (document.fonts && document.fonts.load) {docum

In [19]:
# 印出第一筆資料來觀察 print out the first data to see 
look_up = []
for vid in soup.select(".yt-lockup-video"):
    look_up.append(vid)
    
print(look_up[0])

<div class="yt-lockup yt-lockup-tile yt-lockup-video vve-check clearfix" data-context-item-id="GAhWN9zmEV0" data-visibility-tracking="CEgQ3DAYASITCKrv8Mr_4OQCFZWSwgoduAAFyUDdopjn_caVhBg="><div class="yt-lockup-dismissable yt-uix-tile"><div class="yt-lockup-thumbnail contains-addto"><a aria-hidden="true" class="yt-uix-sessionlink spf-link" data-sessionlink="itct=CEgQ3DAYASITCKrv8Mr_4OQCFZWSwgoduAAFyTIGc2VhcmNoUgVCZXJnZQ" href="/watch?v=GAhWN9zmEV0"><div class="yt-thumb video-thumb"><span class="yt-thumb-simple">
<img alt="" data-ytimg="1" height="138" onload=";window.__ytRIL &amp;&amp; __ytRIL(this)" src="https://i.ytimg.com/vi/GAhWN9zmEV0/hqdefault.jpg?sqp=-oaymwEmCPYBEIoBSFryq4qpAxgIARUAAAAAGAElAADIQj0AgKJDeAGAAQE=&amp;rs=AOn4CLCvCLxnQ_sPizN81vnaJaTk7nLWwA" width="246"/>
<span aria-hidden="true" class="video-time">3:37</span></span></div></a> <span class="thumb-menu dark-overflow-action-menu video-actions">
</span>
<button class="yt-uix-button yt-uix-button-size-small yt-uix-button-de

### 創建一些空清單等等可以裝我們想要的資料
### Create several new list so we can put data we want into them later on

In [20]:
title = []
uploaded = []
watch_counts = []
description = []
link = []

### 讀取每一筆資料，區分出我們要的部分然後存進各類別清單
### Read through the data, filter out the ones we want, then save them into the lists we created
![Title](2702.JPG)

In [21]:
for vid in soup.select(".yt-lockup-video"):
    
    data = vid.select("a[rel='spf-prefetch']") # 找到有我們要的資料區塊 get the section that contains the info we want
    title.append(data[0].get("title")) # 存標題 save the title
    link.append("https://www.youtube.com{}".format(data[0].get("href"))) # 存連結 save the link
    
    time = vid.select(".yt-lockup-meta-info") # 找到有我們要的資料區塊 get the section that contains the info we want
    uploaded.append(time[0].get_text("#").split("#")[0]) # 存上傳時間 save the uploaded time
    watch_counts.append(time[0].get_text("#").split("#")[1]) # 存觀看次數 save the watch counts
    
    disc = vid.select(".yt-lockup-description") # 找到有我們要的資料區塊 get the section that contains the info we want
    try:
        description.append(disc[0].get_text()) # 有找到簡介的存起來 save if there's description
    except:
        description.append("Nan")
        continue # 如果沒有就存一個空值然後跳過 append a Nan value and continue if there's none

print(title[0])
print(uploaded[0])
print(watch_counts[0])
print(description[0])
print(link[0])

Berge - Für die Liebe (Unplugged)
1 年前
觀看次數：2,999,193次
Das neue Berge Album "Für die Liebe" ist ab jetzt überall erhältlich. Als CD, LP, Download und Stream: ...
https://www.youtube.com/watch?v=GAhWN9zmEV0


### 針對取得簡介的部分說明 Watch out for the getting description part
像上圖的頻道連結或播放清單等，有時裡面會沒有文字或沒有這個區塊，所以要跳過避免報錯。<br>
When we look at the picture above, we can see that sometimes the search results contains the channel of the artist or even playlists. And there might be no string in the description or even without the description section. Thus, we need to set up a continue iteration for our loop to avoid errors occur.

## 儲存我們爬取到的資料 Save the data we scraped

In [23]:
import pandas as pd
berge = {'Title':title, 'Uploaded':uploaded, 'Watch_Counts':watch_counts, 'Description':description, 'Link':link}
berge = pd.DataFrame(berge)
berge.head()

,Title,Uploaded,Watch_Counts,Description,Link
0,Berge - Für die Liebe (Unplugged),1 年前,"觀看次數：2,999,193次","Das neue Berge Album ""Für die Liebe"" ist ab je...",https://www.youtube.com/watch?v=GAhWN9zmEV0
1,Berge - Mein Lied (Offizielles Musikvideo),2 個月前,"觀看次數：229,689次","Mein Lied' und das Album ,Für die Liebe' zum V...",https://www.youtube.com/watch?v=OGm3j3Qbipc
2,Berge - Trau Dich (Offizielles Video),2 週前,"觀看次數：72,523次","""Trau Dich"" findest Du auch auf unserem neuen ...",https://www.youtube.com/watch?v=HCdnECKWIZ8
3,Berge - 10.000 Tränen (Live im RAW Berlin),5 年前,"觀看次數：5,208,711次","Das neue Berge Album ""Für die Liebe"" ist ab je...",https://www.youtube.com/watch?v=IS_mantk3tA
4,Berge - Wir sind frei (Videoclip),3 年前,"觀看次數：209,260次",Offizielles Video zu 'Wir sind frei' von Berge...,https://www.youtube.com/watch?v=pZ7HyHKkuuI


In [ ]:
berge.to_csv('berge.csv')

文中若有錯誤還望不吝指正，感激不盡。
Please let me know if there’s any mistake in this article. Thanks for reading.

Reference 參考資料：

[1] [Beautiful Soup 4.2.0](https://www.crummy.com/software/BeautifulSoup/bs4/doc/index.zh.html)

[2] [爬蟲實戰-Youtube](https://hiskio.com/courses/112/lectures/3159)

[3] [Youtube](https://www.youtube.com/results?search_query=Berge)

[4] [IndexError](https://blog.csdn.net/qq_33731081/article/details/84788473)